``` sql

CREATE TABLE input1 (
    txndate DATE,
    status VARCHAR(50)
);

INSERT INTO input1 VALUES ('2022-10-11', NULL);
INSERT INTO input1 VALUES ('2022-10-12', NULL);
INSERT INTO input1 VALUES ('2022-10-13', 'In Stock');
INSERT INTO input1 VALUES ('2022-10-16', 'Out of Stock');
INSERT INTO input1 VALUES ('2022-10-17', NULL);
INSERT INTO input1 VALUES ('2022-10-18', NULL);
INSERT INTO input1 VALUES ('2022-10-19', NULL);

select * from input1;

+------------+--------------+
| txndate    | status       |
+------------+--------------+
| 2022-10-11 | NULL         |
| 2022-10-12 | NULL         |
| 2022-10-13 | In Stock     |
| 2022-10-16 | Out of Stock |
| 2022-10-17 | NULL         |
| 2022-10-18 | NULL         |
| 2022-10-19 | NULL         |
+------------+--------------+


Write an SQL query to display the final status of inventory as 'In Stock' or 'Out of Stock'

expected output:

+------------+---------------+
| txndate    | filled_status |
+------------+---------------+
| 2022-10-11 | NULL          |
| 2022-10-12 | NULL          |
| 2022-10-13 | In Stock      |
| 2022-10-14 | In Stock      |
| 2022-10-15 | In Stock      |
| 2022-10-16 | Out of Stock  |
| 2022-10-17 | Out of Stock  |
| 2022-10-18 | Out of Stock  |
| 2022-10-19 | Out of Stock  |
+------------+---------------+



**Solution:**

``` sql
WITH RECURSIVE cte AS (
    SELECT MIN(txndate) AS tdate FROM input1
    UNION ALL
    SELECT tdate + INTERVAL 1 DAY
    FROM cte
    WHERE tdate + INTERVAL 1 DAY <= (SELECT MAX(txndate) FROM input1)
),
joined AS (
    SELECT cte.tdate AS txndate, i.status
    FROM cte
    LEFT JOIN input1 i ON cte.tdate = i.txndate
),
fill_forward AS (
    SELECT 
        txndate,
        status,
        ROW_NUMBER() OVER (ORDER BY txndate) AS rn
    FROM joined
),
final AS (
    SELECT 
        f1.txndate,
        -- fill nulls using the latest non-null status using correlated subquery
        (
            SELECT f2.status 
            FROM fill_forward f2 
            WHERE f2.rn <= f1.rn AND f2.status IS NOT NULL
            ORDER BY f2.rn DESC
            LIMIT 1
        ) AS filled_status
    FROM fill_forward f1
)

SELECT * FROM final;


---

**Solution-2:**
``` sql
WITH RECURSIVE cte_dates AS (
    SELECT MIN(txndate) AS txndate FROM input1
    UNION ALL
    SELECT txndate + INTERVAL 1 DAY
    FROM cte_dates
    WHERE txndate + INTERVAL 1 DAY <= (SELECT MAX(txndate) FROM input1)
),
cte_input AS (
    SELECT d.txndate, i.status
    FROM cte_dates d
    LEFT JOIN input1 i ON d.txndate = i.txndate
),
fill_forward AS (
    SELECT 
        txndate,
        status,
        status AS filled_status
    FROM cte_input
    WHERE txndate = (SELECT MIN(txndate) FROM input1)

    UNION ALL

    SELECT 
        ci.txndate,
        ci.status,
        CASE 
            WHEN ci.status IS NOT NULL THEN ci.status
            ELSE ff.filled_status
        END AS filled_status
    FROM cte_input ci
    JOIN fill_forward ff ON ci.txndate = ff.txndate + INTERVAL 1 DAY
)
SELECT txndate, filled_status
FROM fill_forward
ORDER BY txndate;
